In [1]:
from autora.cycle.filesystem import FilesystemCycle
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
def ground_truth(x):
    return x + 1
from autora.variable import VariableCollection, Variable
metadata_0 = VariableCollection(
   independent_variables=[Variable(name="x1", allowed_values=range(11))],
   dependent_variables=[Variable(name="y", value_range=(-20, 20))],
   )



In [3]:
from autora.experimentalist.pipeline import make_pipeline
example_experimentalist = make_pipeline(
    [metadata_0.independent_variables[0].allowed_values])

def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(x):
        return ground_truth(x) + rng.normal(0, 0.1, x.shape)
    return runner
example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
example_synthetic_experiment_runner(np.ndarray([1]))



array([1.04339546])

In [4]:

example_theorist = LinearRegression()


In [5]:
from autora.cycle.planner import random_operation_planner
from autora.cycle.result.serializer.yaml_ import YAMLResultCollectionSerializer
import pathlib

cycle = FilesystemCycle(
    metadata=metadata_0,
    theorist=example_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    monitor=lambda data: print(f"Generated {len(data)}-th datum, a new {data[-1].kind}"),
    serializer=YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting")),
    planner=random_operation_planner,
)




In [6]:
for i in range(10):
    try:
        next(cycle)
    except (IndexError, ValueError):
        pass


Generated 1-th datum, a new CONDITION
Generated 2-th datum, a new CONDITION
Generated 3-th datum, a new OBSERVATION
Generated 4-th datum, a new THEORY
Generated 5-th datum, a new OBSERVATION
Generated 6-th datum, a new CONDITION
Generated 7-th datum, a new THEORY
Generated 8-th datum, a new THEORY
Generated 9-th datum, a new CONDITION


In [7]:
new_cycle = FilesystemCycle(
    state = YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting")),
    theorist=example_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    monitor=lambda data: print(f"Generated {len(data)}-th datum, a new {data[-1].kind}"),)
new_cycle.run(num_steps=10)


Generated 27-th datum, a new THEORY
Generated 28-th datum, a new CONDITION
Generated 29-th datum, a new OBSERVATION
Generated 30-th datum, a new THEORY
Generated 31-th datum, a new CONDITION
Generated 32-th datum, a new OBSERVATION
Generated 33-th datum, a new THEORY
Generated 34-th datum, a new CONDITION
Generated 35-th datum, a new OBSERVATION
Generated 36-th datum, a new THEORY


In [ ]:
new_cycle.serializer = YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting"))
new_cycle.dump()
